In [1]:
import pandas as pd
import pdb

In [15]:
def convert_encode_desc(df, col):
    """
    Convert encode desc. of a sample into something more parseable
    """
    df[col] = df[col].str.lower()
    df[col] = df[col].str.replace(', ', '_')
    df[col] = df[col].str.replace(' ', '_')
    df[col] = df[col].str.replace('-', '_')
    
    return df

In [31]:
def convert_biosample_treatments(df, col):
# turn biosample treatments into tuples
    df.loc[df[col].isnull(), col] = ''
    df[col] = df.apply(lambda x: tuple(sorted(x[col].replace(' ', '').split(','))), axis=1)
    return df

In [32]:
df = pd.read_csv("metadata.tsv", sep='\t')

df.index = df['File accession']
df = df.loc[df['Genome annotation'] == 'V29']
df = df[['File accession', 'Experiment accession', \
         'Biosample term name', 'Biosample type', \
         'Technical replicate(s)', 'Biological replicate(s)',
         'Biosample treatments', 'Biosample treatments duration']]
df['classification'] = 'cell_line'
df.loc[df['Biosample type']=='tissue', 'classification'] = 'tissue'
df = convert_encode_desc(df, 'Biosample term name')




In [33]:

# df.head()
# df.loc[df['Biosample term name'] == 'h1'] # 1/23/23 all are h1 (not dif. by manual search)
# df.loc[df['Biosample term name'] == 'h9'] # 1/23/23 all are h1 (not dif. by manual search)

In [34]:
# convert hyphenated cell line names
term_map = pd.read_csv('../lr_bulk/biosamp_term_name_map.tsv', sep='\t',
                       header=None, names=['eid', 'old_name', \
                                           'Biosample treatments', \
                                           'Biosample treatments duration', 'new_name'])
term_map = convert_encode_desc(term_map, 'old_name')
term_map = convert_encode_desc(term_map, 'new_name')
term_map.drop_duplicates(inplace=True)

# git rid of this at some point
df = df.loc[df['Biosample term name'] == 'hl_60']
term_map = term_map.loc[term_map.old_name == 'hl_60']
# turn biosample treatments into tuples
df = convert_biosample_treatments(df, 'Biosample treatments')
term_map = convert_biosample_treatments(term_map, 'Biosample treatments')


# df['Biosample treatments'].str.split(',').astype(tuple).sort_values()


n1 = len(df.index)
print(n1)
df_merge_cols = ['Biosample term name', 'Biosample treatments', 'Biosample treatments duration']
term_map_merge_cols = ['old_name', 'Biosample treatments', 'Biosample treatments duration']

# merge didn't work 
df = df.merge(term_map, how='left', left_on=df_merge_cols, right_on=term_map_merge_cols)
n2 = len(df.index)
print(n2)
# if n1 != n2:
#     print('Duplicated thingies, check for DE samples')
# df.rename({'Biosample term name': 'sample'}, axis=1, inplace=True)
# df.loc[~df.new_name.isnull(), 'sample'] = df.loc[~df.new_name.isnull(), 'new_name']


16
16


In [35]:
# have to deal with hl60s

In [36]:
term_map['Biosample treatments'].values

array([('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate'),
       ('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('',), ('phorbol13-acetate12-myristate',),
       ('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate')],
      dtype=object)

In [37]:
df['Biosample treatments'].values

array([('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate'),
       ('interleukin-13', 'interleukin-4', 'phorbol13-acetate12-myristate'),
       ('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('interferongamma', 'lipopolysaccharide', 'phorbol13-acetate12-myristate'),
       ('phorbol13-acetate12-myristate',),
       ('phorbol13-acetate12-myristate',),
       ('phorbol13-acetate12-my

In [50]:
term_map.loc[term_map.old_name == 'hl_60', term_map_merge_cols]

,old_name,Biosample treatments,Biosample treatments duration
11,hl_60,"lipopolysaccharide, interferon gamma, phorbol ...","120 hour, 24 hour"
12,hl_60,"interleukin-4, phorbol 13-acetate 12-myristate...","120 hour, 24 hour"
13,hl_60,"lipopolysaccharide, interferon gamma, phorbol ...","120 hour, 72 hour"
14,hl_60,NaN,NaN
15,hl_60,phorbol 13-acetate 12-myristate,120 hour
16,hl_60,"lipopolysaccharide, interferon gamma, phorbol ...","120 hour, 12 hour"
17,hl_60,"interleukin-4, phorbol 13-acetate 12-myristate...","120 hour, 72 hour"
18,hl_60,"interleukin-4, phorbol 13-acetate 12-myristate...","120 hour, 12 hour"


In [37]:
term_map.head()

,old_name,new_name
0,a673,a673
1,caco_2,caco2
2,calu3,calu3
3,gm12878,gm12878
6,gm23338,pgp1
